## Finding Roots of a Non-Linear Equation

Consider the equation: $5e^{-x} +x - 5 = 0$ . Suppose we want to solve it with accuracy tolerance $\epsilon = 10^{-6}$.

### Exercise 1
Solve using binary search. How many iterations are required?

### Exercise 2
Solve using relaxation. How many iterations are required?

### Exercise 3
Solve using Newton's Method. How many iterations are required?

### Exercise 4
Solve using Secant Method. How many iterations are required?

## Fun with Circuits

### Physics background

![Newman's Exercise 6.5](rescap.png)

Consider the above circuit. Suppose the voltage $V_+$ is time-varying and sinusoidal of the form $V_+ = x_+ \exp(i \omega t)$ with $x_+$ a constant.

The resistors in the circuit can be treated using Ohm's law.
For the capacitors the charge $Q$ and voltage $V$ across them are related by the capacitor law $Q=CV$, where $C$ is the capacitance.
Differentiating both sides of this expression gives the current $I$ flowing in on one side of the capacitor and out on the other:
\begin{equation}
  I = \frac{dQ}{dt} = C \frac{dV}{dt}.
\end{equation}
Now assume the voltages at the points labeled 1, 2, and 3 are of the form $V_1 = x_1 \exp(i \omega t)$, $V_2 = x_2 \exp(i \omega t)$, and $V_3 = x_3 \exp(i \omega t)$.
If you add up the currents using Kirchoff's law that at a junction the sum of the currents in equals the sum of the currents out, you can find that the constants $x_1$,
$x_2$, and $x_3$ satisfy the equations
\begin{align*}
  \left( {1\over R_1} + {1\over R_4} + i\omega C_1 \right) x_1 - i\omega C_1 x_2
  & = {x_+\over R_1}\,, \\
  - i\omega C_1 x_1 + \left( {1\over R_2} + {1\over R_5} + i\omega C_1 + i\omega C_2 \right) x_2 - i\omega C_2 x_3
                                                                                           & = {x_+\over R_2}\,, \\
  - i\omega C_2 x_2 + \left( {1\over R_3} + {1\over R_6} + i\omega C_2 \right) x_3
  & = {x_+\over R_3}\,.
\end{align*}
This is a linear system of equations for three complex numbers, $x_1$, $x_2$, and $x_3$.

### Exercise 5

We will be solving the above linear system of equations in the form $A x = b$, where $x$ is the vector $(x_1 x_2 x_3$ and $b$ is the vector composed of the right-hand sides of the equations above.

Write a function that takes as input the list of resistance values ($R_1$ to $R_6$) and the list of capacitances ($C_1$ and $C_2$), and returns (as numpy.array) the matrix $A$. 
Write another function that takes as input the list of resistance values and the value of $x+$, and returns (as numpy.array) the vector $b$.

### Exercise 6

Use Gaussian Elimination with partial pivoting (see the code fragment below) to solve for $x_1$, $x_2$, and $x_3$. Assume the following:
    \begin{align*}
      R_1 &= R_3 = R_5 = 1\,\mathrm{k}\Omega, \\
      R_2 &= R_4 = R_6 = 2\,\mathrm{k}\Omega, \\
      C_1 &= 1\,\mu\mathrm{F},\qquad C_2 = 0.5\,\mu\mathrm{F}, \\
      x_+ &= 3\,\mathrm{V},\qquad \omega = 1000~\text{rad/s}.
    \end{align*}

Have your program calculate and print, at $t=0$, the amplitudes of the three voltages $|V_1|$, $|V_2|$, and $|V_3|$ and their phases (i.e. the phases of the coefficients $x_1, x_2, x_3$) in degrees.

*Notice that the matrix for this problem has complex elements. You will need to define a complex array to hold it, but your routine should be able to work with real or complex arguments.*

*Hint: the built-in abs() will compute the magnitude, and numpy.angle() will compute the phase of a complex number. You could also use polar and phase from the cmath package.*

In [19]:
import numpy as np
def GaussElim(A_in, v_in, pivot=False):
    """Implement Gaussian Elimination. This should be non-destructive for input
    arrays, so we will copy A and v to temporary variables
    It is actually clever to modify this function, rather than copy it and
    modify it in a separate function. The reason is that if you find a bug in
    the common parts, then you only need to fix it once.
    IN:
    A_in [np.array], the matrix to pivot and triangularize
    v_in [np.array], the RHS vector
    pivot [bool, default-False]: user decides if we pivot or not.
    OUT:
    x, the vector solution of A_in x = v_in """
    # copy A and v to temporary variables using copy command
    A = np.copy(A_in)
    v = np.copy(v_in)
    N = len(v)

    for m in range(N):
        if pivot:  # This is where I modify GaussElim
            # compare the mth element to all other mth elements below
            ZeRow = m
            for mm in range(m+1, N):
                if abs(A[mm, m]) > abs(A[ZeRow, m]):
                    ZeRow = mm  # I could swap everytime I find a hit, but that
                    # would be a lot of operations. Instead, I just take note
                    # of which row emerges as the winner

            if ZeRow != m:  # now we know if and what to swap
                A[ZeRow, :], A[m, :] = np.copy(A[m, :]), np.copy(A[ZeRow, :])
                v[ZeRow], v[m] = np.copy(v[m]), np.copy(v[ZeRow])

        # Divide by the diagonal element
        div = A[m, m]
        A[m, :] /= div
        v[m] /= div

        # Now subtract from the lower rows
        for i in range(m+1, N):
            mult = A[i, m]
            A[i, :] -= mult*A[m, :]
            v[i] -= mult*v[m]

    # Backsubstitution
    # create an array of the same type as the input array
    x = np.empty(N, dtype=v.dtype)
    for m in range(N-1, -1, -1):
        x[m] = v[m]
        for i in range(m+1, N):
            x[m] -= A[m, i]*x[i]
    return x

def PartialPivot(A_in, v_in):
    """ see textbook p. 222) """
    return GaussElim(A_in, v_in, True)

### Exercise 6


Plot the (real) voltages $|V_1|$, $|V_2|$, and $|V_3|$ as a function of time for two periods
of the oscillations. You should have 3 lines on the same plot, with labelled axes and a nice legend.

*Hint: about 256 points should be enough to obtain a nice plot.*